In [1]:
from pymongo import MongoClient

In [2]:
uri = "mongodb://localhost:27017"
db_name = "mtg_decks_db"
client = MongoClient(uri)
db = client[db_name]

In [11]:
pipeline = [
    {"$project": {"all_cards": {"$concatArrays": [
        "$maindeck", "$sideboard"]}, "name": 1}},
    {"$unwind": "$all_cards"},
    {"$group": {"_id": {"_id": "$_id", "card_name": "$all_cards",
                        "name": "$name"}, "count": {"$sum": 1}}},
    {"$group": {"_id": "$_id._id", "name": {"$first": "$_id.name"},
                "cards": {"$push": {"name": "$_id.card_name", "count": "$count"}}}}
]

decks = list(db.decks.aggregate(pipeline))
decks

[{'_id': ObjectId('6614a6ebbc0fae7824b1c7c3'),
  'name': 'Familiars',
  'cards': [{'name': 'Last Breath', 'count': 2},
   {'name': 'Counterspell', 'count': 2},
   {'name': 'Destroy Evil', 'count': 3},
   {'name': 'Mortuary Mire', 'count': 1},
   {'name': 'Snap', 'count': 4},
   {'name': 'Prohibit', 'count': 1},
   {'name': "God-Pharaoh's Faithful", 'count': 4},
   {'name': 'Lórien Revealed', 'count': 3},
   {'name': 'Idyllic Beachfront', 'count': 1},
   {'name': 'Ephemerate', 'count': 2},
   {'name': 'Ash Barrens', 'count': 4},
   {'name': 'Merchant Scroll', 'count': 2},
   {'name': 'Brainstorm', 'count': 2},
   {'name': 'Archaeomancer', 'count': 3},
   {'name': 'Sea Gate Oracle', 'count': 3},
   {'name': 'Mulldrifter', 'count': 4},
   {'name': 'Murmuring Mystic', 'count': 2},
   {'name': 'Echoing Truth', 'count': 1},
   {'name': "Nature's Chant", 'count': 2},
   {'name': 'Hydroblast', 'count': 4},
   {'name': 'Azorius Chancery', 'count': 4},
   {'name': 'Plains', 'count': 2},
   {'nam

In [12]:
personal_pool = {}
for card in db.personal_pool.find({}, {"name": 1, "count": 1, "_id": 0}):
    personal_pool[card["name"]] = card["count"]

personal_pool

{'_____ Goblin': 4,
 'Abrade': 4,
 'Acorn Harvest': 4,
 'All That Glitters': 4,
 'Alpine Meadow': 4,
 'Ancient Den': 4,
 'Ancient Stirrings': 4,
 'Annul': 4,
 "Apostle's Blessing": 4,
 'Arboreal Grazer': 3,
 'Archaeomancer': 4,
 'Architects of Will': 4,
 'Arctic Treeline': 3,
 'Ardent Elementalist': 4,
 'Arms of Hadar': 4,
 'Ash Barrens': 4,
 'Augur of Bolas': 4,
 'Aura Flux': 1,
 'Axebane Guardian': 5,
 'Azorius Chancery': 4,
 'Azure Fleet Admiral': 4,
 'Barren Moor': 4,
 'Behold the Multiverse': 5,
 'Benevolent Bodyguard': 5,
 'Bladeback Sliver': 2,
 'Blood Celebrant': 4,
 'Blood Fountain': 4,
 'Blue Elemental Blast': 4,
 'Bojuka Bog': 4,
 'Boulderbranch Golem': 4,
 'Brainstorm': 4,
 'Breath Weapon': 4,
 'Brinebarrow Intruder': 4,
 'Bring the Ending': 4,
 'Brute Force': 3,
 'Burning Prophet': 4,
 'Cabal Ritual': 4,
 'Cast Down': 4,
 'Cast into the Fire': 4,
 'Cave of Temptation': 1,
 'Chain Lightning': 4,
 'Chromatic Sphere': 4,
 'Chromatic Star': 4,
 'Compulsive Research': 2,
 'Cont

In [15]:
usable_decks = []
for deck in decks:
    can_build = True
    for card in deck["cards"]:
        if card["name"] not in personal_pool or personal_pool[card["name"]] < card["count"]:
            can_build = False
            break
    if can_build:
        usable_decks.append(deck["_id"])

In [16]:
for deck_id in usable_decks:
    deck = db.decks.find_one({"_id": deck_id})
    print(f"{deck["name"]} ({deck["_id"]})")

Jeskai Ephemerate (660e289b9904164478622168)
Izzet Skred (6614a4c0bc0fae7824b1c578)
Grixis Affinity (6614a682bc0fae7824b1c75d)
Golgari Gardens (6614a4febc0fae7824b1c5bc)
Dimir Faeries (660e296b99041644786221d4)
Grixis Affinity (660e2a87990416447862228e)
Azorius Affinity (660e23f29904164478621e4f)
Azorius Affinity (660e26289904164478621fde)
Kuldotha Red (6614a869bc0fae7824b1c95a)
Azorius Affinity (660e25719904164478621f61)
Dimir Faeries (6613e21c89bd2415fd534026)
Rakdos Madness Burn (661afcf98397a502be1d6e01)
Rakdos Madness (660e232d9904164478621db3)
Caw-Gates (660e2afa99041644786222d9)
Caw-Gates (660e259d9904164478621f7e)
Dimir Terror (660e23f79904164478621e53)
Caw-Gates (6613e20289bd2415fd53400a)
Boros Synthesizer (660e29dc990416447862221f)
Jeskai Affinity (660e29fc9904164478622236)
Azorius Affinity (660e25939904164478621f75)
Golgari Garden (661afdc48397a502be1d6e3c)
Azorius Affinity (6614a539bc0fae7824b1c607)
Dimir Control (66169bfb2eb712be577eedce)
Caw-Gates (6614a699bc0fae7824b1c77